In [ ]:
#importing neccessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import datetime
from sklearn.preprocessing import LabelEncoder
from xgboost.sklearn import XGBClassifier
import warnings
warnings.filterwarnings('ignore')


In [ ]:
sessions=pd.read_csv('../input/airbnb-recruiting-new-user-bookings/sessions.csv.zip')
countries=pd.read_csv('../input/airbnb-recruiting-new-user-bookings/countries.csv.zip')
age_gender=pd.read_csv('../input/airbnb-recruiting-new-user-bookings/age_gender_bkts.csv.zip')
submission=pd.read_csv('../input/airbnb-recruiting-new-user-bookings/sample_submission_NDF.csv.zip')

In [ ]:
#Loading the Data
train= pd.read_csv('../input/airbnb-recruiting-new-user-bookings/train_users_2.csv.zip')
test = pd.read_csv('../input/airbnb-recruiting-new-user-bookings/test_users.csv.zip')


In [ ]:
train.signup_method.unique()

In [ ]:
train.affiliate_channel.unique()

* Arama Motoru Optimizasyonu (SEO), Web sitelerinin arama motorlarında daha iyi performans göstermesi için yapılan çalışmaların tümüne verilen isimdir.

* Remarketing en yalın tanımı ile, sitenizi ziyaret eden kişilerin etiketlenerek, sitenizden ayrıldıktan sonra ziyaret ettikleri diğer sitelerde tekrar reklamınızı görmelerini sağlamaktır.

In [ ]:
train.affiliate_provider.unique()

In [ ]:
train.first_affiliate_tracked.unique()

In [ ]:
train.first_device_type.unique()

In [ ]:
train.first_browser.unique()

In [ ]:
train.signup_app.unique()

In [ ]:
countries.sort_values(by='distance_km')

In [ ]:
a=train['country_destination'].value_counts() 
a=a.drop(['NDF','other'],axis=0) 
df_value_counts = pd.DataFrame(a)
df_value_counts = a.reset_index() 
df_value_counts.columns = ['country_destination', 'value_counts'] # change column names 
mapdata= pd.merge(df_value_counts, countries,how= 'inner' , on='country_destination')
mapdata['poppercent']=mapdata['value_counts']/mapdata['value_counts'].sum() 
mapdata['text']=['United States', 'France', 'Italy','United Kingdom', 'Spanish','Canada' ,'German', 'Dutch','Australia', 'Brazil' ] 

In [ ]:
# Create a world map to show distributions of users 
import folium
from folium.plugins import MarkerCluster
#empty map
world_map= folium.Map(tiles="cartodbpositron")
marker_cluster = MarkerCluster().add_to(world_map)
#for each coordinate, create circlemarker of user percent
for i in range(len(mapdata)):
        lat = mapdata.iloc[i]['lat_destination']
        long = mapdata.iloc[i]['lng_destination']
        radius=5
        popup_text = """Country : {}<br>
                    %of Users : {}<br>"""
        popup_text = popup_text.format(mapdata.iloc[i]['country_destination'],
                                   mapdata.iloc[i]['poppercent']
                                   )
        folium.CircleMarker(location = [lat, long], radius=radius, popup= popup_text, fill =True).add_to(marker_cluster)
#show the map
world_map

Australya(AU),İngiltere(GB),Kanada(CA) ve Amerika(US) ingilizce konuşuyor. İnsanlar dillerinin konuşulduğu yerleri öncelikli olarak tercih etmiş olabilir mi? Yoksa Amerikaya veya Amerikaya yakın yerlerin tercih edilmesi daha olası mı?

In [ ]:
a=train.groupby('country_destination').count().sort_values(by='id',ascending=True)
a.id


İnsanlar öncelikli olarak Amerika içinde seyehat etmişler. En çok seyehat edilen(adını bildiğimiz) ikinci ülke Fransa, üçüncü ülke ise İtalya. Yani dil ve uzaklık tercih edilme açısından bir kriterdir diyemeyiz. Language_levenshtein_distance ile country destination arasındaki ilişkiyi gözlemleyebileceğimiz bir plot çizerek bu hipotez kontrol edilebilir. Ama yorumlarımda bir değişiklik olmayacaktır.

In [ ]:
countries.dtypes

In [ ]:
age_gender.drop(age_gender.index[0],inplace=True)
age_gender

In [ ]:
age_gender.country_destination.unique()

In [ ]:
age_gender.age_bucket.unique()

In [ ]:
agebtw0n19= age_gender[(age_gender.age_bucket=='0-4') | (age_gender.age_bucket=='5-9') | (age_gender.age_bucket=='10-14')|(age_gender.age_bucket=='15-19')]
agebtw49n34= age_gender[(age_gender.age_bucket=='45-49') | (age_gender.age_bucket=='40-44') | (age_gender.age_bucket=='35-39') | (age_gender.age_bucket=='30-34')]
agebtw99n50= age_gender[(age_gender.age_bucket=='95-99') | (age_gender.age_bucket=='90-94') | (age_gender.age_bucket=='85-89') | (age_gender.age_bucket=='80-84') | (age_gender.age_bucket=='75-79') | (age_gender.age_bucket=='70-74') | (age_gender.age_bucket=='70-74') | (age_gender.age_bucket=='65-69') | (age_gender.age_bucket=='60-64') | (age_gender.age_bucket=='55-59') | (age_gender.age_bucket=='50-54') ]
agebtw20n29= age_gender[(age_gender.age_bucket=='25-29') | (age_gender.age_bucket=='20-24')]


In [ ]:
import seaborn as sns
ax = sns.boxplot(x=agebtw20n29["population_in_thousands"])

In [ ]:
import seaborn as sns
ax = sns.boxplot(x=agebtw99n50["population_in_thousands"])

In [ ]:
plt.figure(figsize=(20,5))
import seaborn as sns
age_gender.sort_values("age_bucket", ascending=False,inplace=True)
colors=sns.color_palette()
sns.stripplot(x="age_bucket",y="population_in_thousands",data=age_gender,jitter=True,hue='country_destination',palette='pastel')

60 yaşından büyükler yaşları ilerledikçe daha sakin lokasyonları tercih etmiş. Ama genel olarak yaş, gidilen ülke ve populasyon büyüklüğü arasındadoğrudan bir ilişki yok gibi görünüyor. Ama yıl bilgisi sadece 2015 yılına ait. Chi squre test katagorik değişkenlerimiz arasında bir ilişki olup olmadığını test etmemizi sağlayacaktır.

In [ ]:
actionanditsdetail=pd.crosstab(age_gender.age_bucket, age_gender.country_destination,margins=False)
from scipy.stats import chi2_contingency 

# defining the table 

stat, p, dof, expected = chi2_contingency(actionanditsdetail) 

# interpret p-value 
alpha = 0.05
print("p value is " + str(p)) 
if p <= alpha: 
	print('Dependent (reject H0)') 
else: 
	print('Independent (H0 holds true)')

In [ ]:
actionanditsdetail=pd.crosstab(age_gender.country_destination, age_gender.gender,margins=False)
from scipy.stats import chi2_contingency 

# defining the table 

stat, p, dof, expected = chi2_contingency(actionanditsdetail) 

# interpret p-value 
alpha = 0.05
print("p value is " + str(p)) 
if p <= alpha: 
	print('Dependent (reject H0)') 
else: 
	print('Independent (H0 holds true)')


age_gender ve countries datalarını incelediğimde train datasıyla birleştirebileceğim bir değişken gözlemleyemedim. İnsanların tatillerini geçirebileceği yer seçimlerinde gidilecek ülkenin uzaklığı, o ülkede konuşulan dilin ingilizce veya ingilizceye yakın bir dil ailesine sahip oluşu,cinsiyet, kişinin yaşı ve yaşa bağlı olarak gidilen yerin nufus yogunluğu arasında anlamlı bir ilişki gözlemleyemedim. Ama tatile gidilecek döneme bağlı olarak insanlar şehir dışına çıkmayı veya Amerika içinde gezmeyi tercih etmiş olabilirler.

In [ ]:
sessions=sessions.sort_values('user_id')
sessions.rename(columns={"user_id": "id"},inplace=True)
sessions.sort_values(by='id',inplace=True)
sessions = sessions[(sessions['secs_elapsed'].notnull()) & (sessions['secs_elapsed'] > 0.0) ]  
sessions[(sessions['action_detail']=='booking')]

Sadece 3 kişinin hareketleri booking ile sonuçlanmış. 3 ü de windows kullanıcısı olarak görünüyor ama bence eylemlerin rezervasyon ile sonuçlanması detaylı araştırma gerektireceği için bilgisayardan islem yapılıyor olması daha olası. Daha sonra device type ı bilgisayar olup olmamasına göre yeniden isimlendirebilirim. train datasına sitede geçirilen toplam süre bilgisi ekleyip bu veriyi scale edebilirim. Bu iki yöntemin modelimi iyileştirip iyileştirmediğine bakabilirim.

In [ ]:
groupedid=pd.DataFrame(sessions.groupby(['id'])['secs_elapsed'].sum())
groupedid['hour_elapsed']=groupedid['secs_elapsed'].div(3600).round(decimals=0)
groupedid.drop('secs_elapsed',axis=1,inplace=True)
groupedid.reset_index(inplace=True)
groupedid[(groupedid['id'] == '6udv3scuxe') | (groupedid['id'] == 'yxf0sm9sbw') | (groupedid['id'] == 'nttj7g9av6')] 

In [ ]:
groupedid.hour_elapsed.describe()

Rezervasyon yapan insanların sitede geçirdikleri toplam süre için diger insanların geçirdikleri sürenin %75 inden fazla zaman geçirdikleri söylenebilir. Süre bakımından üst çeyreklikte bulunuyorlar.

In [ ]:
actionndevicetype=pd.crosstab(sessions.action, sessions.device_type,margins=False)
from scipy.stats import chi2_contingency 

# defining the table 

stat, p, dof, expected = chi2_contingency(actionndevicetype) 

# interpret p-value 
alpha = 0.05
print("p value is " + str(p)) 
if p <= alpha: 
	print('Dependent (reject H0)') 
else: 
	print('Independent (H0 holds true)')

Device type ve action arasında bir ilişki var gibi görünüyor. % 95 güven aralığıyla kullanıcıların hareketleriyle kullandıkları alet arasında anlamlı bir ilişki vardır diyebiliriz.

Train datasıyla birleştirebileceğimherhengi bir özellik gözlemleyemedim. Ancak modellerimin doğrulugunu arttırmak için train datasının kolonlarıyla oynayıp datayı yeniden şekillendirebileceğimi düşündüm.

# TRAIN DATA EDA

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
print(train.country_destination.value_counts())
sns.countplot(train.country_destination)

Sınıflar eşit dağılmamış. Azınlık sınıfların kötü tahmin edilmesine sebep olacak bir problem. 'resampling' teknikleri deneyerek bir denge sağlayabileceğimi düşünüyorum.

# MODELLING

1. MISSINGNESS

In [ ]:
import pandas as pd
#Loading the Data again
train= pd.read_csv('../input/airbnb-recruiting-new-user-bookings/train_users_2.csv.zip')
test = pd.read_csv('../input/airbnb-recruiting-new-user-bookings/test_users.csv.zip')


train ve test i yenilemek istediğimde notebookun başına gitmemek için buraya taşıdım bu kodları.

In [ ]:
#missing data
total = train.isnull().sum().sort_values(ascending=False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

In [ ]:
train['first_affiliate_tracked'] = train['first_affiliate_tracked'].fillna('Unknown')
test['first_affiliate_tracked'] = test['first_affiliate_tracked'].fillna('Unknown')

In [ ]:
import seaborn as sns
sns.boxplot(x=train.age)

In [ ]:
train.age.describe()

In [ ]:
import numpy as np
#train=train[(train.age > 14) & (train.age < 110)]# modele sadece 14 yaşından büyük ve 110 yaşından küçükleri dahil edebilirim.
#test=test[(test.age > 14) & (test.age < 110)]
train[(train.age < 14) & (train.age > 110)]=np.nan
test[(test.age < 14) & (test.age > 110)]=np.nan
train['age'].fillna(train['age'].mean(), inplace=True)
test['age'].fillna(train['age'].mean(), inplace=True)

14 yaşından küçüklerin aileleriyle seyehat edeceklerini varsayarak 14 yaşından küçükleri modele dahil etmedim. Önceki araştırmalarımda küçük yaştaki yolcuların tercihleriyle büyük yaştaki yolcuların tercihleri arasında anlamlı bir fark olmadığını gördüm. Ayrıca 110 yaşından büyük insanların yaşlarını incelediğimde anlamsız yaşlarda(2014 gibi) insanlar gördüm. O nedenle sadece 110 yaşından küçük ve 14 yaşından büyük insanlar için sınıflandırma yapmaya karar verdim.

Mean imputation yerine başka yöntemler de denenebilirdi. Ama age değişkeni mean imputaion için uygun bir değişkendi. Ayrıca pratik bir yöntem olduğunu düşündüm. Modeli iyileştirip iyileştirmeyeceğini görmek için başka imputation yöntemleri de denenebilir. 14 yaşından küçük çocukları dataya dahil ederek çocuklu ebebynlerin seyehat tercihleri hakkında bir fikir sahibi olunabilirdi belki ama age_gender datasına baktıgımda yaş gurupları ve country destinationları arasında bir ilişki gözlemleyememiştim. 14 yaşından küçükleri modele dahil etmek modelim üzerinde büyük farklaryaratmayacaktır. Ek olarak, 110 yaşından büyük yaşları incelediğimde 2014, 1929 gibi yaşlar gözlemledim. Bunlar missing gibi görünmesede aslında kayıp bilgiler. 800 küşür kişiye ait bilgileri modelden atmaktansa onları da missing olarak kabul edip imputation uygulamak modellerimi iyileştirebilir. Sonraki aşamalarda modelimi ileriye taşımak adına deneyeceğim bir yöntem olacaktır.

In [ ]:
#Converting below columns as categories for plotting in graphs
categorical_features = [
    'affiliate_channel',
    'affiliate_provider',
    'first_affiliate_tracked',
    'first_browser',
    'first_device_type',
    'gender',
    'language',
    'signup_app',
    'signup_method',
    'signup_flow'
]

for categorical_feature in categorical_features:
    train[categorical_feature] = train[categorical_feature].astype('category')
for categorical_feature in categorical_features:
    test[categorical_feature] = test[categorical_feature].astype('category')    

signup_flow değişkeni sayısal bir değermiş gibi dursa da önceki notebooklarımdan birinde grafiksel olarak bu değişkene kategorik demenin daha doğru olacağını düşünmüştüm. Yarışmaya dair açıklamaların yazdığı sayfada bunun siteye kayıt olunmadan önceki sayfa olduguna dair bir açıklama vardı. Bu iki durumu göz önünde bulundurunca signup_flow değişkenine kategorik gibi davranmam gerektiğine karar verdim.

In [ ]:
train['date_account_created'] = pd.to_datetime(train['date_account_created'])
train['date_first_booking'] = pd.to_datetime(train['date_first_booking'])
train['timestamp_first_active'] = pd.to_datetime(train['timestamp_first_active'], format='%Y%m%d%H%M%S')
train['timestamp_first_active'] = pd.to_datetime(train['timestamp_first_active']).dt.date
test['date_account_created'] = pd.to_datetime(test['date_account_created'])
test['timestamp_first_active'] = pd.to_datetime(test['timestamp_first_active'], format='%Y%m%d%H%M%S')
test['timestamp_first_active'] = pd.to_datetime(test['timestamp_first_active']).dt.date


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# Use seaborn style defaults and set the default figure size

df = train.groupby(['date_first_booking'])['country_destination'].count().reset_index()
df.dropna(axis=0,inplace=True)
import plotly.express as px

fig = px.line(df, x='date_first_booking', y="country_destination")
fig.show()


Her yıl temmuz ve agustos aylarında ilk rezervasyon sayıları artmış. 2011'den 2014' e kadar genel bir artış var. 2104 yılının temmuz ayından mayıs 2015' e kadar azalışa geçmiş. 2015'in temmuz ve agustos aylarına dair bir bilgi mevcut değil.

Rezervasyonlar üzerinde zaman etkisi gözardı edilemez bu nedenle tarih bilgisi bulunduran verileri sınıflandırma algoritmasına dahil etmek modellerimi iyileştirebilir.

In [ ]:
train.drop(['date_first_booking'], axis=1,inplace=True)
test.drop(['date_first_booking'], axis=1,inplace=True)

In [ ]:
#date_account_created

dac = np.vstack(train.date_account_created.astype(str).apply(lambda x: list(map(int, x.split('-')))).values)
train['dac_year'] = dac[:,0]
train['dac_month'] = dac[:,1]
train['dac_day'] = dac[:,2]
train.drop(['date_account_created'], axis=1,inplace=True)
#timestamp_first_active
tfa = np.vstack(train.timestamp_first_active.astype(str).apply(lambda x: list(map(int, x.split('-')))).values)
train['tfa_year'] = tfa[:,0]
train['tfa_month'] = tfa[:,1]
train['tfa_day'] = tfa[:,2]
train.drop(['timestamp_first_active'], axis=1,inplace=True)
#date_account_created

dac = np.vstack(test.date_account_created.astype(str).apply(lambda x: list(map(int, x.split('-')))).values)
test['dac_year'] = dac[:,0]
test['dac_month'] = dac[:,1]
test['dac_day'] = dac[:,2]
test.drop(['date_account_created'], axis=1,inplace=True)
#timestamp_first_active
tfa = np.vstack(test.timestamp_first_active.astype(str).apply(lambda x: list(map(int, x.split('-')))).values)
test['tfa_year'] = tfa[:,0]
test['tfa_month'] = tfa[:,1]
test['tfa_day'] = tfa[:,2]
test.drop(['timestamp_first_active'], axis=1,inplace=True)

Sitede gerçekleştirilen ilk eylemin zamanının ve siteye ilk kayıt olunan tarihin zamanlarının gün, ay, yıl bilgilerini kullanarak yeni değişkenler üretmek modelimi iyileştirse de çok büyük farklar yaratmadı. Sürekli yeni yöntemler denediğim için bu bilgilerin modelden atılmış halini sizlerle paylaşmadım. Ayrıca date_first_booking değişkenini veriden attım. Çünkü zaten ilk booking yapılan zaman test datasında yok. Aslında country_destination'ı tahmin etmeden önce date_first_booking' i de tahmin etmek gerekiyor olabilir. Çünkü yeni bir müşterinin ne zaman geleceği, geldiğinde gitmek isteyebileceği ilk yeri etkileyen bir faktör. Yani burada aslında zamana bağlı/dönemsel bir takım etkiler de söz konusu. Aynı anda birden fazla değişkeni tahminleyen yöntem ya da önce yeni müşterilerin gelecekleri tarihleri tahminleyip sonra 'gelseler, nereye giderlerdi?' sorusunu sormak mantıklı olabilir belki.

# Computation for the Booking Destination

# LABEL ENCODING

Sınıflandırma algoritmaları kullanacağım için one-hot encoding yerine label encoding kullanmayı tercih ettim. Eğer bu veri regresyona uygun olsaydı one-hot encoding uygulardım. Çünkü label encoding yaptıgımda bir değişkene ait sınıf çok fazla olduğunda bir sınıfın diğerinden daha önemli olduğunu varsayabiliyor modelimiz. Sayısal bir büyüklük olarak algılıyor sınıflara verilen sayıları. Bu da modellerimizin doğruluğunu olumsuz etkileyebilir. Ancak sınıflandırma algoritmalarında böyle bir sorun söz konusu değil. One-hot-encoding uygulamak da yalnış olmazdı ancak şimdilik buna gerek olduğunu düşünmüyorum.

In [ ]:
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder() 
  
train['gender']= le.fit_transform(train['gender'])
train['signup_method']= le.fit_transform(train['signup_method']) 
train['first_affiliate_tracked']= le.fit_transform(train['first_affiliate_tracked']) 
train['signup_method']= le.fit_transform(train['signup_method']) 
train['language']= le.fit_transform(train['language'])
train['affiliate_channel']= le.fit_transform(train['affiliate_channel'])
train['affiliate_provider']= le.fit_transform(train['affiliate_provider'])
train['signup_app']= le.fit_transform(train['signup_app'])
train['first_device_type']= le.fit_transform(train['first_device_type'])
train['first_browser']= le.fit_transform(train['first_browser'])
train['signup_flow']= le.fit_transform(train['signup_flow'])

In [ ]:
le = LabelEncoder() 
  
test['gender']= le.fit_transform(test['gender'])
test['signup_method']= le.fit_transform(test['signup_method']) 
test['first_affiliate_tracked']= le.fit_transform(test['first_affiliate_tracked']) 
test['signup_method']= le.fit_transform(test['signup_method']) 
test['language']= le.fit_transform(test['language'])
test['affiliate_channel']= le.fit_transform(test['affiliate_channel'])
test['affiliate_provider']= le.fit_transform(test['affiliate_provider'])
test['signup_app']= le.fit_transform(test['signup_app'])
test['first_device_type']= le.fit_transform(test['first_device_type'])
test['first_browser']= le.fit_transform(test['first_browser'])
test['signup_flow']= le.fit_transform(test['signup_flow'])

In [ ]:
train.country_destination.replace('NDF',0,inplace=True)
train.country_destination.replace('US',1,inplace=True)
train.country_destination.replace('other',2,inplace=True)
train.country_destination.replace('FR',3,inplace=True)
train.country_destination.replace('CA',4,inplace=True)
train.country_destination.replace('GB',5,inplace=True)
train.country_destination.replace('ES',6,inplace=True)
train.country_destination.replace('IT',7,inplace=True)
train.country_destination.replace('PT',8,inplace=True)
train.country_destination.replace('NL',9,inplace=True)
train.country_destination.replace('DE',10,inplace=True)
train.country_destination.replace('AU',11,inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
y=train['country_destination']
X=train.drop(['country_destination','id'],axis=1)
from imblearn.combine import SMOTETomek
# transform the dataset
smotetomek = SMOTETomek(sampling_strategy='auto')

# split the dataset into train and test sets
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.30, random_state=1,shuffle=True,stratify=y)
X_train, y_train = smotetomek.fit_resample(X_train1, y_train1)

SMOTETomek- resampling yaparak sınıfları dengeliyorum. Train-test ayrımını 'stratify=y' ile yaptığımda  'Stratified Samplimg' diğer adıyla tabakalı örnekleme yaparak country_destination değişkenini temsil eden her bir alt sınıfı train ve test verilerine dahik ettiğime ein oluyorum.

> Using Stratified Sampling technique ensures that there will be selection from each sub-groups and prevents the chance of omitting one sub-group leading to sampling bias

In [ ]:
x=pd.DataFrame(X_train)
Y=pd.DataFrame(y_train)
result = pd.concat([x, Y], axis=1, join='inner')
sns.countplot(result.country_destination)

Resampling yapmak sınıfları daha eşit dağıttı. 

In [ ]:
target_names = ['NDF', 'US', 'other', 'FR', 'CA', 'GB', 'ES', 'IT', 'PT', 'NL','DE', 'AU']
#Classifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


rf=RandomForestClassifier()

rf.fit(X_train,y_train)

y_predrf=rf.predict(X_test1)
print(classification_report(y_test1, y_predrf, target_names=target_names))

In [ ]:
feature_names=list(X_train1.columns.values.tolist()) 

In [ ]:
import pandas as pd
feature_imp = pd.Series(rf.feature_importances_,index=feature_names).sort_values(ascending=False)
feature_imp


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
# Creating a bar plot
sns.barplot(x=feature_imp, y=feature_imp.index)
# Add labels to your graph
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.legend()
plt.show()

**** 0.69 submission score (smoote oversampling kullandığımda).
    Farklı resampling algoritmalarını deneyerek modelimi en iyi haline getirmeye çalışasagım.
Smootetomek fonsiyonu scorumu biraz düşürsede classlarımı çok güzel dengeledi. Bu nedenle modelim için SMOTE'dan daha iyi bir seçenek olduğunu düşünüyorum.

In [ ]:
pred_country={0:"NDF", 1:"US", 2:"other", 3:"FR", 4:"CA", 5:"GB", 6:"ES", 7:"IT", 8:"PT", 9:"DE", 10:"NL", 11:"AU"}

In [ ]:
from xgboost.sklearn import XGBClassifier
xgb = XGBClassifier()                  

xgb.fit(X_train,y_train)


y_predxgb=xgb.predict(X_test1)

print(classification_report(y_test1, y_predxgb, target_names=target_names))


Random Forest a kıyasla daha iyi bir classification raporum olduğunu söyleyebilirim. Resampling uyguladıktan ve XGBoost yaptıktan sonra submission scorum 0.59 a düştü. Ancak Başlangıçta country_destination değişkenin içindeki veriler eşit örneğe sahip değildi. Bu nedenle veride en sık bulunan iki sınıfı çok iyi tahmin ederken diğer sınıfları neredeyse hiç tahmin etmiyordu modelim. Bu problemi çözmek için 'resampling' yöntemlerini araştırdım. Undersampling yöntemlerini deneyecek kadar büyük bir veri setine sahip olmadığım için oversampling yöntemlerine yöneldim. Ancak bu yöntemleri denedikten sonra bile 'balanced' bir y değişkenine sahip olamadığımı gözlemledim. Sonra over ve under sampling yöntemlerinin kombinasyonu olarak tanımlanabilecek yöntemlere yöneldim. Sınıflarımı en iyi dengeleyen yöntem [SMOTEomek](http://imbalanced-learn.readthedocs.io/en/stable/api.html#module-imblearn.combine) oldu. 

In [ ]:
predictionsxgb=xgb.predict(test.drop(['id'],axis=1))

In [ ]:
resultsxgb=[]
for i in predictionsxgb:
    resultsxgb.append(pred_country[i])

In [ ]:
#my_submissionxgb = pd.DataFrame({'id': test.id, 'country':resultsxgb})
#my_submissionxgb.to_csv('submissionxgb.csv', index=False)

In [ ]:
from sklearn.neural_network import MLPClassifier
#Generate prediction using Neural Net

mlp = MLPClassifier(activation='identity', solver='sgd',learning_rate='adaptive', alpha=0.0001, batch_size='auto')
mlp.fit(X_train,y_train)
predsmlp = mlp.predict(X_test1)
from sklearn import metrics
print(classification_report(y_test1, predsmlp, target_names=target_names))

Mlp accuracy olarak XGB den daha yüksek bir accuray ye sahip olsa da azınlık sınıfları tahminlerken çuvalladı. Rezervasyon yapmayacak kullanıcıları iyi tahmin etti ancak yapanların nereye gideceğini hiç tahmin edemedi. Submission scorumsa 0.67'ye yükseldi. Bazı parametreleri değiştirmenin modelimin üzerinde nasıl bir etki bırakacağını gözlemlemek istiyorum.

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
parameters = {'solver': ['lbfgs','sgd'], 'max_iter': [1000,1100,1200,1300,1400,1500,1600,1700,1800,1900,2000 ], 'alpha': 10.0 ** -np.arange(1, 10), 'hidden_layer_sizes':np.arange(10, 15), 'random_state':[0,1,2,3,4,5,6,7,8,9]}
mlpgridsearch = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1)
mlpgridsearch.fit(X_train,y_train)
predsgridmlp = mlpgridsearch.predict(X_test1)
from sklearn import metrics
print(classification_report(y_test1, predsgridmlp, target_names=target_names))

In [ ]:
from sklearn.naive_bayes import ComplementNB cnb = ComplementNB() cnb.fit(X_train, y_train) y_predcnb=cnb.predict(X_test1)

from sklearn import metrics

print(classification_report(y_test1, y_predcnb, target_names=target_names))

Tablo değerleri random forest  ve mlp ile kıyasladığımda baya kötü. Ancak tüm sınıflara (destinationlara) yapılacak ilk rezervasyonları mlp'den daha iyi tahmin etti. Random Forest ı geçemediği için bu algoritmayı eledim.

In [ ]:
import pandas as pd import numpy as np from sklearn.preprocessing import LabelEncoder from xgboost.sklearn import XGBClassifier

#Loading the Data again train= pd.read_csv('../input/airbnb-recruiting-new-user-bookings/train_users_2.csv.zip') test = pd.read_csv('../input/airbnb-recruiting-new-user-bookings/test_users.csv.zip') train['first_affiliate_tracked'] = train['first_affiliate_tracked'].fillna('Unknown') test['first_affiliate_tracked'] = test['first_affiliate_tracked'].fillna('Unknown') train['date_account_created'] = pd.to_datetime(train['date_account_created']) train['timestamp_first_active'] = pd.to_datetime(train['timestamp_first_active'], format='%Y%m%d%H%M%S') train['timestamp_first_active'] = pd.to_datetime(train['timestamp_first_active']).dt.date test['date_account_created'] = pd.to_datetime(test['date_account_created']) test['timestamp_first_active'] = pd.to_datetime(test['timestamp_first_active'], format='%Y%m%d%H%M%S') test['timestamp_first_active'] = pd.to_datetime(test['timestamp_first_active']).dt.date train.drop(['date_first_booking'], axis=1,inplace=True) test.drop(['date_first_booking'], axis=1,inplace=True)

#date_account_created

dac = np.vstack(train.date_account_created.astype(str).apply(lambda x: list(map(int, x.split('-')))).values) train['dac_year'] = dac[:,0] train['dac_month'] = dac[:,1] train['dac_day'] = dac[:,2] train.drop(['date_account_created'], axis=1,inplace=True)

#timestamp_first_active tfa = np.vstack(train.timestamp_first_active.astype(str).apply(lambda x: list(map(int, x.split('-')))).values) train['tfa_year'] = tfa[:,0] train['tfa_month'] = tfa[:,1] train['tfa_day'] = tfa[:,2] train.drop(['timestamp_first_active'], axis=1,inplace=True)

#date_account_created

dac = np.vstack(test.date_account_created.astype(str).apply(lambda x: list(map(int, x.split('-')))).values) test['dac_year'] = dac[:,0] test['dac_month'] = dac[:,1] test['dac_day'] = dac[:,2] test.drop(['date_account_created'], axis=1,inplace=True)

#timestamp_first_active tfa = np.vstack(test.timestamp_first_active.astype(str).apply(lambda x: list(map(int, x.split('-')))).values) test['tfa_year'] = tfa[:,0] test['tfa_month'] = tfa[:,1] test['tfa_day'] = tfa[:,2] test.drop(['timestamp_first_active'], axis=1,inplace=True) import numpy as np train[(train.age < 14) & (train.age > 110)]=np.nan train['age'].fillna(train['age'].mean(), inplace=True)

test[(test.age < 14) & (test.age > 110)]=np.nan test['age'].fillna(train['age'].mean(), inplace=True)

#Converting below columns as categories for plotting in graphs categorical_features = [ 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'first_browser', 'first_device_type', 'gender', 'language', 'signup_app', 'signup_method', 'signup_flow' ]

for categorical_feature in categorical_features: train[categorical_feature] = train[categorical_feature].astype('category') for categorical_feature in categorical_features: test[categorical_feature] = test[categorical_feature].astype('category')

#One-hot-encoding features ohe_feats = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser'] for f in ohe_feats: train_dummy = pd.get_dummies(train[f], prefix=f) train_cont= train.drop([f], axis=1) train = pd.concat((train_cont, train_dummy), axis=1)

#One-hot-encoding features ohe_feats = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser'] for f in ohe_feats: test_dummy = pd.get_dummies(test[f], prefix=f) test_cont= test.drop([f], axis=1) test = pd.concat((test_cont, test_dummy), axis=1)

#Splitting train and test

from sklearn.model_selection import train_test_split y=train['country_destination'] X=train.drop(['country_destination','id'],axis=1) from imblearn.combine import SMOTETomek

transform the dataset
smotetomek = SMOTETomek(sampling_strategy='auto')

split the dataset into train and test sets
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.30, random_state=1,shuffle=True) X_train, y_train = smotetomek.fit_resample(X_train1, y_train1)

#Classifier

xgb = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1, importance_type='gain', interaction_constraints=None, learning_rate=0.3, max_delta_step=0, max_depth=6, min_child_weight=1, monotone_constraints=None, n_estimators=25, n_jobs=0, num_parallel_tree=1, objective='multi:softprob', random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None, seed=0, subsample=0.5, tree_method=None, validate_parameters=False, verbosity=None) xgb.fit(X_train, y_train) target_names = ['NDF', 'US', 'other', 'FR', 'CA', 'GB', 'ES', 'IT', 'PT', 'NL','DE', 'AU'] y_predxgb=xgb.predict(X_test1)

from sklearn.metrics import classification_report print(classification_report(y_test1, y_predxgb, target_names=target_names))

Sadece İtalya destinationının presicion değeri yüksek çıktı. Farklı parametler ve one hot encoding kullanarak yeniden XGboost denediğimde accuracy 0.2 arttı ama bu model pek doğru sınıflandırma yapmadı.

#Eklemek istediklerim:

Önceki denemelerimde timestamp_first_active ve date_account_created değişkenlerini üzerlerinde hiçbir değişiklik yapmadan silmiştim. Ama bu iki değişkeni bölüp gün,ay,yıl şeklinde yeni değişkenler üretmek submission scorumu neredeyse hiç değiştirmedi. En yüksek scorelarımı sadece date_first_booking değişkenini kullandığımda elde etmiştim. Bu değişkeni gün,ay,yıl olacak şekilde 3 e parçalayıp veri setine eklemiştin ve ardından kategorik değişkenlerime one-hot-endoding uygulamıştım, scorelarım çok anlamlı bir şekilde değişmişti ancak 'Test' verisinde date_first_booking değişkeni tamamen boştu ayrıca azınlıkta olan sınıfların tahmin edilebilirliği oldukça düşüktü hatta bazı sınıflar hiç tahmin edilemiyordu. Bu nedenle yarışmaya herhangi bir yükleme yapamadım. Yeni kullanıcıların ilk rezervasyon lokasyonları zamana bağlı değişebilen bir şey. Belki[ bu dataya uygun bir time series ](https://otexts.com/fpp2/hierarchical.html)algoritması bulunabilir ve dataya o şekilde yaklaşılabilir.